In [1]:
import requests
from bs4 import BeautifulSoup
import re
import json
import io
import pandas as pd
import yfinance as yf

In [2]:
tickers1 = ['siemens', 'deutsche-telekom', 'allianz', 'merck-kgaa','deutsche-post']

In [3]:
tickers2 = ['SIE.DE', 'DTE.DE', 'ALV.DE', 'MRK.DE', 'DHL.DE']

In [4]:
ratios = ['pe-ratio', 'ps-ratio', 'pb-ratio']

In [5]:
eps = 'eps'

In [6]:
url = 'https://companiesmarketcap.com/'

In [7]:
# Define the date range
start_date = pd.to_datetime('2010-01-01').date()
end_date = pd.to_datetime('2023-07-01').date()

In [8]:
def get_ratio(ticker, ratio, url, start_date, end_date):
    url_temp = url + ticker + '/' + ratio + '/'
    # Make an HTTP request to fetch the HTML content
    response = requests.get(url_temp)
    html_content = response.text
    # Parse the HTML content with BeautifulSoup
    soup = BeautifulSoup(html_content, 'html.parser')
    # Find all script elements with type="text/javascript"
    script_elements = soup.find_all('script', {'type': 'text/javascript'})
    # Convert script element to string
    script_text = str(script_elements[0].string)
    # Find index of "data"
    idx1 = script_text.find("[{") + 1
    # Find index of "var"
    idx2 = script_text.find("}]") + 1
    # Extract data array
    json_string = script_text[idx1 : idx2]
    # Convert the string to a DataFrame
    df = pd.read_json(io.StringIO('[' + json_string.replace('}\n{', '},{') + ']'))
    # Rename columns
    df.columns = ['Date', ratio]
    # Convert 'Date' column to datetime
    df['Date'] = pd.to_datetime(df['Date'], unit='s', origin='unix')
    # Convert 'Date' column to date
    df['Date'] = df['Date'].dt.date
    # Filter rows based on the specified date range
    #df_filtered = df[(df['Date'] >= start_date) & (df['Date'] <= end_date)]
    return df

In [9]:
def get_price(ticker, start_date, end_date):
    ticker = yf.Ticker(ticker)
    df = ticker.history(start=start_date, end=end_date, interval="1d")
    df = df.reset_index()
    df['Date'] = df['Date'].dt.date
    return df

In [10]:
ticker1 = tickers1[0]
ticker2 = tickers2[0]
# Get price from yfinance
df0 = get_price(ticker=ticker2, start_date=start_date, end_date=end_date)
# Get ratio from url
df1 = get_ratio(ticker=ticker1, ratio=ratios[0], url=url, start_date=start_date, end_date=end_date)
df2 = get_ratio(ticker=ticker1, ratio=ratios[1], url=url, start_date=start_date, end_date=end_date)
df3 = get_ratio(ticker=ticker1, ratio=ratios[2], url=url, start_date=start_date, end_date=end_date)
# Merge filtered DataFrames on the 'Date' column
df = pd.merge(df0, df1, on='Date', how='outer').sort_values(by='Date')
df = pd.merge(df, df2, on='Date', how='left')
df = pd.merge(df, df3, on='Date', how='left')
df['Ticker'] = ticker1
# Fill na valua by previous value, remove first line and reset index
df = df.fillna(method='ffill')
df = df[(df['Date'] >= start_date) & (df['Date'] <= end_date)]
df = df.reset_index(drop=True)
# Write date to csv file
filename = f"{ticker1}.csv"
df.to_csv(filename, index=False)
df

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,pe-ratio,ps-ratio,pb-ratio,Ticker
0,2010-01-04,39.453718,39.898526,39.362319,39.892433,2734897.0,0.0,0.0,21.092000,0.762592,1.938798,siemens
1,2010-01-05,39.849778,40.148345,39.593861,39.916801,2584717.0,0.0,0.0,21.092000,0.762592,1.938798,siemens
2,2010-01-06,39.880244,40.130068,39.752286,40.063042,2166413.0,0.0,0.0,21.092000,0.762592,1.938798,siemens
3,2010-01-07,40.032572,40.702831,39.709632,40.538311,3336865.0,0.0,0.0,21.092000,0.762592,1.938798,siemens
4,2010-01-08,40.733292,40.964836,40.325047,40.794228,2634386.0,0.0,0.0,21.092000,0.762592,1.938798,siemens
...,...,...,...,...,...,...,...,...,...,...,...,...
3444,2023-06-26,157.399994,157.699997,155.320007,155.479996,1312767.0,0.0,0.0,19.691292,1.515738,1.998793,siemens
3445,2023-06-27,156.399994,158.039993,155.979996,157.559998,1065494.0,0.0,0.0,19.691292,1.515738,1.998793,siemens
3446,2023-06-28,158.940002,159.639999,150.399994,152.720001,2094689.0,0.0,0.0,19.691292,1.515738,1.998793,siemens
3447,2023-06-29,151.460007,152.839996,149.720001,150.580002,2239915.0,0.0,0.0,19.691292,1.515738,1.998793,siemens


In [11]:
ticker1 = tickers1[1]
ticker2 = tickers2[1]
# Get price from yfinance
df0 = get_price(ticker=ticker2, start_date=start_date, end_date=end_date)
# Get ratio from url
df1 = get_ratio(ticker=ticker1, ratio=ratios[0], url=url, start_date=start_date, end_date=end_date)
# Fix value PE at 30-09-2017
target_date = pd.to_datetime('2017-09-30')
index_of_target_date = df1.loc[df1['Date'] == target_date].index[0]
df1.at[index_of_target_date, 'pe-ratio'] = df1.at[index_of_target_date-1, 'pe-ratio']
# Get ratio from url
df2 = get_ratio(ticker=ticker1, ratio=ratios[1], url=url, start_date=start_date, end_date=end_date)
df3 = get_ratio(ticker=ticker1, ratio=ratios[2], url=url, start_date=start_date, end_date=end_date)
# Merge filtered DataFrames on the 'Date' column
df = pd.merge(df0, df1, on='Date', how='outer').sort_values(by='Date')
df = pd.merge(df, df2, on='Date', how='left')
df = pd.merge(df, df3, on='Date', how='left')
df['Ticker'] = ticker1
# Fill na valua by previous value, remove first line and reset index
df = df.fillna(method='ffill')
df = df[(df['Date'] >= start_date) & (df['Date'] <= end_date)]
df = df.reset_index(drop=True)
# Write date to csv file
filename = f"{ticker1}.csv"
df.to_csv(filename, index=False)
df

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\ops\array_ops.py:73: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  result = libops.scalar_compare(x.ravel(), y, op)


,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,pe-ratio,ps-ratio,pb-ratio,Ticker
0,2010-01-04,4.826971,4.953136,4.817626,4.953136,14257860.0,0.0,0.0,128.625000,0.707311,1.070129,deutsche-telekom
1,2010-01-05,4.948465,4.953138,4.871364,4.911083,14077418.0,0.0,0.0,128.625000,0.707311,1.070129,deutsche-telekom
2,2010-01-06,4.831645,4.833982,4.770899,4.789590,28100490.0,0.0,0.0,128.625000,0.707311,1.070129,deutsche-telekom
3,2010-01-07,4.766226,4.770899,4.663425,4.698472,21799244.0,0.0,0.0,128.625000,0.707311,1.070129,deutsche-telekom
4,2010-01-08,4.707817,4.728844,4.668565,4.724171,20945576.0,0.0,0.0,128.625000,0.707311,1.070129,deutsche-telekom
...,...,...,...,...,...,...,...,...,...,...,...,...
3444,2023-06-26,19.528000,19.733999,19.493999,19.695999,6407184.0,0.0,0.0,5.658228,1.014428,1.057418,deutsche-telekom
3445,2023-06-27,19.830000,19.968000,19.712000,19.906000,0.0,0.0,0.0,5.658228,1.014428,1.057418,deutsche-telekom
3446,2023-06-28,20.000000,20.155001,19.927999,20.030001,8299496.0,0.0,0.0,5.658228,1.014428,1.057418,deutsche-telekom
3447,2023-06-29,20.045000,20.094999,19.840000,19.930000,4255420.0,0.0,0.0,5.658228,1.014428,1.057418,deutsche-telekom


In [12]:
ticker1 = tickers1[2]
ticker2 = tickers2[2]
# Get price from yfinance
df0 = get_price(ticker=ticker2, start_date=start_date, end_date=end_date)
# Get ratio from url
df1 = get_ratio(ticker=ticker1, ratio=ratios[0], url=url, start_date=start_date, end_date=end_date)
df2 = get_ratio(ticker=ticker1, ratio=ratios[1], url=url, start_date=start_date, end_date=end_date)
df3 = get_ratio(ticker=ticker1, ratio=ratios[2], url=url, start_date=start_date, end_date=end_date)
# Merge filtered DataFrames on the 'Date' column
df = pd.merge(df0, df1, on='Date', how='outer').sort_values(by='Date')
df = pd.merge(df, df2, on='Date', how='left')
df = pd.merge(df, df3, on='Date', how='left')
df['Ticker'] = ticker1
# Fill na valua by previous value, remove first line and reset index
df = df.fillna(method='ffill')
df = df[(df['Date'] >= start_date) & (df['Date'] <= end_date)]
df = df.reset_index(drop=True)
# Write date to csv file
filename = f"{ticker1}.csv"
df.to_csv(filename, index=False)
df

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,pe-ratio,ps-ratio,pb-ratio,Ticker
0,2010-01-04,45.138680,45.699056,45.035857,45.519119,1676777.0,0.0,0.0,9.154412,0.706716,1.292624,allianz
1,2010-01-05,45.498552,46.110342,45.395732,45.657925,1742206.0,0.0,0.0,9.154412,0.706716,1.292624,allianz
2,2010-01-06,45.832732,46.161760,45.627088,46.012669,1362139.0,0.0,0.0,9.154412,0.706716,1.292624,allianz
3,2010-01-07,45.714482,45.981821,45.421441,45.483135,1926360.0,0.0,0.0,9.154412,0.706716,1.292624,allianz
4,2010-01-08,45.848145,45.848145,44.562875,45.236355,2425290.0,0.0,0.0,9.154412,0.706716,1.292624,allianz
...,...,...,...,...,...,...,...,...,...,...,...,...
3444,2023-06-26,209.800003,209.800003,207.699997,209.199997,532054.0,0.0,0.0,10.264393,0.759921,1.461468,allianz
3445,2023-06-27,210.149994,211.449997,209.649994,211.300003,640049.0,0.0,0.0,10.264393,0.759921,1.461468,allianz
3446,2023-06-28,212.000000,212.800003,210.750000,211.649994,575057.0,0.0,0.0,10.264393,0.759921,1.461468,allianz
3447,2023-06-29,211.949997,212.100006,211.050003,211.050003,474146.0,0.0,0.0,10.264393,0.759921,1.461468,allianz


In [13]:
ticker1 = tickers1[3]
ticker2 = tickers2[3]
# Get price from yfinance
df0 = get_price(ticker=ticker2, start_date=start_date, end_date=end_date)
# Get ratio from url
df1 = get_ratio(ticker=ticker1, ratio=ratios[0], url=url, start_date=start_date, end_date=end_date)
df2 = get_ratio(ticker=ticker1, ratio=ratios[1], url=url, start_date=start_date, end_date=end_date)
df3 = get_ratio(ticker=ticker1, ratio=ratios[2], url=url, start_date=start_date, end_date=end_date)
# Merge filtered DataFrames on the 'Date' column
df = pd.merge(df0, df1, on='Date', how='outer').sort_values(by='Date')
df = pd.merge(df, df2, on='Date', how='left')
df = pd.merge(df, df3, on='Date', how='left')
df['Ticker'] = ticker1
# Fill na valua by previous value, remove first line and reset index
df = df.fillna(method='ffill')
df = df[(df['Date'] >= start_date) & (df['Date'] <= end_date)]
df = df.reset_index(drop=True)
# Write date to csv file
filename = f"{ticker1}.csv"
df.to_csv(filename, index=False)
df

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,pe-ratio,ps-ratio,pb-ratio,Ticker
0,2010-01-04,26.572761,26.780616,26.438269,26.576838,752504.0,0.0,0.0,38.785717,NaN,1.488928,merck-kgaa
1,2010-01-05,26.527928,26.560534,26.238563,26.381210,488178.0,0.0,0.0,38.785717,NaN,1.488928,merck-kgaa
2,2010-01-06,26.409742,26.576839,26.140752,26.364910,511138.0,0.0,0.0,38.785717,NaN,1.488928,merck-kgaa
3,2010-01-07,26.324149,26.972167,26.324149,26.919186,1088580.0,0.0,0.0,38.785717,NaN,1.488928,merck-kgaa
4,2010-01-08,26.919188,27.053680,26.328227,26.646122,730274.0,0.0,0.0,38.785717,NaN,1.488928,merck-kgaa
...,...,...,...,...,...,...,...,...,...,...,...,...
3444,2023-06-26,153.000000,154.399994,152.000000,152.600006,255508.0,0.0,0.0,23.026846,3.467976,3.013593,merck-kgaa
3445,2023-06-27,152.250000,152.600006,147.199997,147.199997,603984.0,0.0,0.0,23.026846,3.467976,3.013593,merck-kgaa
3446,2023-06-28,147.550003,150.699997,147.350006,150.500000,419786.0,0.0,0.0,23.026846,3.467976,3.013593,merck-kgaa
3447,2023-06-29,150.449997,150.899994,147.899994,149.600006,384773.0,0.0,0.0,23.026846,3.467976,3.013593,merck-kgaa


In [14]:
ticker1 = tickers1[4]
ticker2 = tickers2[4]
# Get price from yfinance
df0 = get_price(ticker=ticker2, start_date=start_date, end_date=end_date)
# Get ratio from url
df1 = get_ratio(ticker=ticker1, ratio=ratios[0], url=url, start_date=start_date, end_date=end_date)
df2 = get_ratio(ticker=ticker1, ratio=ratios[1], url=url, start_date=start_date, end_date=end_date)
df3 = get_ratio(ticker=ticker1, ratio=ratios[2], url=url, start_date=start_date, end_date=end_date)
# Merge filtered DataFrames on the 'Date' column
df = pd.merge(df0, df1, on='Date', how='outer').sort_values(by='Date')
df = pd.merge(df, df2, on='Date', how='left')
df = pd.merge(df, df3, on='Date', how='left')
df['Ticker'] = ticker1
# Fill na valua by previous value, remove first line and reset index
df = df.fillna(method='ffill')
df = df[(df['Date'] >= start_date) & (df['Date'] <= end_date)]
df = df.reset_index(drop=True)
# Write date to csv file
filename = f"{ticker1}.csv"
df.to_csv(filename, index=False)
df

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,pe-ratio,ps-ratio,pb-ratio,Ticker
0,2010-01-04,13.550000,13.980000,13.550000,13.980,3020790.0,0.0,0.0,25.443396,NaN,1.970697,deutsche-post
1,2010-01-05,13.935000,14.135000,13.830000,14.000,2735783.0,0.0,0.0,25.443396,NaN,1.970697,deutsche-post
2,2010-01-06,13.980000,14.230000,13.925000,14.155,2595800.0,0.0,0.0,25.443396,NaN,1.970697,deutsche-post
3,2010-01-07,14.110000,14.195000,14.020000,14.125,2907679.0,0.0,0.0,25.443396,NaN,1.970697,deutsche-post
4,2010-01-08,14.140000,14.525000,14.060000,14.460,5462896.0,0.0,0.0,25.443396,NaN,1.970697,deutsche-post
...,...,...,...,...,...,...,...,...,...,...,...,...
3436,2023-06-26,43.435001,43.514999,42.619999,43.000,1812008.0,0.0,0.0,10.613300,0.578296,1.588590,deutsche-post
3437,2023-06-27,43.314999,43.590000,43.174999,43.500,2197942.0,0.0,0.0,10.613300,0.578296,1.588590,deutsche-post
3438,2023-06-28,43.709999,44.415001,43.645000,44.305,1739606.0,0.0,0.0,10.613300,0.578296,1.588590,deutsche-post
3439,2023-06-29,44.435001,44.639999,44.150002,44.195,1470995.0,0.0,0.0,10.613300,0.578296,1.588590,deutsche-post


# Combine prices of 5 stocks in 1 file

In [15]:
tickers2 = ['SIE.DE', 'DTE.DE', 'ALV.DE', 'MRK.DE', 'DHL.DE']
tickers1 = ['siemens', 'deutsche-telekom', 'allianz', 'merck-kgaa','deutsche-post']

In [16]:
SIE = pd.read_csv("siemens.csv")
DTE = pd.read_csv("deutsche-telekom.csv")
ALV = pd.read_csv("allianz.csv")
MRK = pd.read_csv("merck-kgaa.csv")
DHL = pd.read_csv("deutsche-post.csv")

In [17]:
price_df = SIE[['Date', 'Close']]

In [18]:
new_df = price_df.rename(columns={'Close': 'SIE'})

In [19]:
new_df

,Date,SIE
0,2010-01-04,39.892433
1,2010-01-05,39.916801
2,2010-01-06,40.063042
3,2010-01-07,40.538311
4,2010-01-08,40.794228
...,...,...
3444,2023-06-26,155.479996
3445,2023-06-27,157.559998
3446,2023-06-28,152.720001
3447,2023-06-29,150.580002


In [20]:
new_df = pd.merge(new_df, DTE[['Date', 'Close']], on='Date', how='left')
new_df = new_df.rename(columns={'Close': 'DTE'})

In [21]:
new_df

,Date,SIE,DTE
0,2010-01-04,39.892433,4.953136
1,2010-01-05,39.916801,4.911083
2,2010-01-06,40.063042,4.789590
3,2010-01-07,40.538311,4.698472
4,2010-01-08,40.794228,4.724171
...,...,...,...
3444,2023-06-26,155.479996,19.695999
3445,2023-06-27,157.559998,19.906000
3446,2023-06-28,152.720001,20.030001
3447,2023-06-29,150.580002,19.930000


In [22]:
new_df = pd.merge(new_df, ALV[['Date', 'Close']], on='Date', how='left')
new_df = new_df.rename(columns={'Close': 'ALV'})

In [23]:
new_df

,Date,SIE,DTE,ALV
0,2010-01-04,39.892433,4.953136,45.519119
1,2010-01-05,39.916801,4.911083,45.657925
2,2010-01-06,40.063042,4.789590,46.012669
3,2010-01-07,40.538311,4.698472,45.483135
4,2010-01-08,40.794228,4.724171,45.236355
...,...,...,...,...
3444,2023-06-26,155.479996,19.695999,209.199997
3445,2023-06-27,157.559998,19.906000,211.300003
3446,2023-06-28,152.720001,20.030001,211.649994
3447,2023-06-29,150.580002,19.930000,211.050003


In [24]:
new_df = pd.merge(new_df, MRK[['Date', 'Close']], on='Date', how='left')
new_df = new_df.rename(columns={'Close': 'MRK'})
new_df

,Date,SIE,DTE,ALV,MRK
0,2010-01-04,39.892433,4.953136,45.519119,26.576838
1,2010-01-05,39.916801,4.911083,45.657925,26.381210
2,2010-01-06,40.063042,4.789590,46.012669,26.364910
3,2010-01-07,40.538311,4.698472,45.483135,26.919186
4,2010-01-08,40.794228,4.724171,45.236355,26.646122
...,...,...,...,...,...
3444,2023-06-26,155.479996,19.695999,209.199997,152.600006
3445,2023-06-27,157.559998,19.906000,211.300003,147.199997
3446,2023-06-28,152.720001,20.030001,211.649994,150.500000
3447,2023-06-29,150.580002,19.930000,211.050003,149.600006


In [25]:
new_df = pd.merge(new_df, DHL[['Date', 'Close']], on='Date', how='left')
new_df = new_df.rename(columns={'Close': 'DHL'})
new_df

,Date,SIE,DTE,ALV,MRK,DHL
0,2010-01-04,39.892433,4.953136,45.519119,26.576838,13.980
1,2010-01-05,39.916801,4.911083,45.657925,26.381210,14.000
2,2010-01-06,40.063042,4.789590,46.012669,26.364910,14.155
3,2010-01-07,40.538311,4.698472,45.483135,26.919186,14.125
4,2010-01-08,40.794228,4.724171,45.236355,26.646122,14.460
...,...,...,...,...,...,...
3444,2023-06-26,155.479996,19.695999,209.199997,152.600006,43.000
3445,2023-06-27,157.559998,19.906000,211.300003,147.199997,43.500
3446,2023-06-28,152.720001,20.030001,211.649994,150.500000,44.305
3447,2023-06-29,150.580002,19.930000,211.050003,149.600006,44.195


In [26]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3449 entries, 0 to 3448
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Date    3449 non-null   object 
 1   SIE     3449 non-null   float64
 2   DTE     3449 non-null   float64
 3   ALV     3449 non-null   float64
 4   MRK     3449 non-null   float64
 5   DHL     3437 non-null   float64
dtypes: float64(5), object(1)
memory usage: 188.6+ KB


In [27]:
new_df = new_df.fillna(method='ffill')
new_df

,Date,SIE,DTE,ALV,MRK,DHL
0,2010-01-04,39.892433,4.953136,45.519119,26.576838,13.980
1,2010-01-05,39.916801,4.911083,45.657925,26.381210,14.000
2,2010-01-06,40.063042,4.789590,46.012669,26.364910,14.155
3,2010-01-07,40.538311,4.698472,45.483135,26.919186,14.125
4,2010-01-08,40.794228,4.724171,45.236355,26.646122,14.460
...,...,...,...,...,...,...
3444,2023-06-26,155.479996,19.695999,209.199997,152.600006,43.000
3445,2023-06-27,157.559998,19.906000,211.300003,147.199997,43.500
3446,2023-06-28,152.720001,20.030001,211.649994,150.500000,44.305
3447,2023-06-29,150.580002,19.930000,211.050003,149.600006,44.195


In [28]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3449 entries, 0 to 3448
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Date    3449 non-null   object 
 1   SIE     3449 non-null   float64
 2   DTE     3449 non-null   float64
 3   ALV     3449 non-null   float64
 4   MRK     3449 non-null   float64
 5   DHL     3449 non-null   float64
dtypes: float64(5), object(1)
memory usage: 188.6+ KB


In [29]:
new_df.to_csv('prices.csv', index=False)